<a href="https://colab.research.google.com/github/anitamezzetti/financial_big_data/blob/main/HienVersion_FinBigData_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
student1 = "Anita Mezzetti"
student2 = "Hien Lê"

Complementary stuff:
* Drive Folder to save data, notebooks and models

Note: project has been linked to [Github](https://github.com/anitamezzetti/financial_big_data)!

In [2]:
from google.colab import drive
drive.mount('/content/drive') # make sure to add the .ipynb files to the Colab Notebooks folder first if not already done

Mounted at /content/drive


In [3]:
!ls "/content/drive/My Drive/Colab Notebooks" # checking the content of the folder

FinBigData_1.ipynb


In [4]:
root = "/content/drive/My Drive/Colab Notebooks" # define name of folder

### Main part

In [8]:
!pip install arrow

     |████████████████████████████████| 51kB 2.5MB/s 


In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# other libraries here
import requests
import arrow

In [ ]:
# The following snippet gets data for the 4 countries from a proxy API, but it seems that the database in this API is not up-to-date
# jh_data = requests.get('https://covid19api.herokuapp.com/confirmed')

# res_dict = jh_data.json()["locations"]
# df_res = pd.json_normalize(res_dict)
# df_res = df_res.drop(columns=["country_code", "latest", "province", "coordinates.latitude", "coordinates.longitude"])

# countries_of_interest = ["US", "Switzerland", "Japan", "China"]

# df_china = df_res[df_res.country == "China"].set_index("country")
# df_china = pd.DataFrame(df_china.sum(axis=0), columns=["China"]).T

# df = df_res[df_res.country.isin(countries_of_interest[:3])].set_index("country")
# df = pd.concat([df, df_china])

In [32]:
countries_of_interest = ["USA", "CHE", "JPN", "CHN"]

endpoint = "https://covidapi.info/api/v1" # endpoint that provides JHU data
start_date = "2020-02-25"
end_date = arrow.utcnow().shift(days=-1).format("YYYY-MM-DD") # yesterday
dfs = {}
for country in countries_of_interest:
  query = f"{endpoint}/country/{country}/timeseries/{start_date}/{end_date}"
  res = requests.get(query).json()
  df = pd.DataFrame(res["result"])
  df.index = df["date"]
  df.to_csv(f"{root}/{country}.csv")
  dfs[country] = df

In [34]:
dfs["JPN"]

,confirmed,date,deaths,recovered
date,,,,
2020-02-25,173,2020-02-25,3,22
2020-02-26,192,2020-02-26,3,22
2020-02-27,218,2020-02-27,4,22
2020-02-28,236,2020-02-28,5,22
2020-02-29,245,2020-02-29,6,32
...,...,...,...,...
2020-11-29,147515,2020-11-29,2057,121735
2020-11-30,148962,2020-11-30,2076,127289
2020-12-01,150976,2020-12-01,2109,125304
